# Create a file with the peaks and trough from all the GP LCs

Python 3.7

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime # Get the current date and time

#--------------------------------------------------------60
# Import my functions

%cd /Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa

import mytoolsSNe

# Go back where this notebook is located:
%cd /Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/Codes/github/MLSNeLC

#--------------------------------------------------------60
code_created_by = 'Arturo_Avelino'
# On date: 2019.06.25 (yyyy.mm.dd)
code_name = 'Extract_peaks_trough_GPLCs.ipynb'
code_version = '0.0.8'
code_last_update = '2019.06.29'

/Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa
/Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/Codes/github/MLSNeLC


In [2]:
# mytoolsSNe.index2day(70)

In [3]:
# mytoolsSNe.is_number('5')

In [4]:
# mytoolsSNe.sn_name('sn1999ee__U_45_B_1_CSP_J_GP_mean_sigma_Filled_norma.dat')

In [5]:
##############################################################################80

### USER

In [6]:

 # info to just write in the output text file
band = 'J'

#---------------------------------------------------------60

# List with the name of the SNe to extract their features
filename = 'list_SNe.txt'

# Location of 'filename' and also of the GP LC files.
dirfile = '/Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/data/data_PCA/'

list_sne = np.genfromtxt(dirfile+filename, dtype=['S60'])

#---------------------------------------------------------60

dir_save_output = '/Users/arturo/Dropbox/Research/Statistics/ML/\
Projects/RAISIN_ML/data/data_PCA/'

-----

## Automatic

### Useful functions

Main function to find the peak and trough in the GP LCs

In [7]:
def find_peak_trough(lcgp_np, feature, phase_range):
    """
    Function to find the peak or trough in a Gaussian-Process light curve (LC).
    By "find" I mean to determine the phase, magnitude, err_magnitude, and
    index in the array, of the peak or trough.
    
    - lcgp_np (numpy array with 3 columns): 
        - column 1: phase
        - column 2: magnitude
        - column 3: err_magnitude
        
    - feature (string): Options = ('peak', 'trough').
        This parameter helps to define whether use 'min' or 'max' functions to
        search for the peaks or the trough, respetively.
        If feature='peak' then it is used the 'min' function to search for the
        -brightest- magnitude. And if feature='trough' then 'max' function is used
        to search for the -dimmest- magnitude in the specified phase range of 
        the LC.
        
    - phase_range (list with 2 elements): 
        
    This function works in either normalized, absolute mag, or apparent 
    magnitude LCs. Also, it works for LCs of any band (e.g, B, V, I, Y, J, H, K)
        
    """
    
    # Define a numpy array of magnitudes from the specified phase range
    mag_search_np = lcgp_np[mytoolsSNe.day2index(phase_range[0]):mytoolsSNe.day2index(phase_range[1]),1]

    #---------------------------------------------------
    #  Find the peak or trough
    
    if feature=='peak':
        
        # Find the -brightest- magnitude in the specified phase range
        mag_feat = min(mag_search_np)
        
        index_feat_int = mag_search_np.argmin()
        
    if feature=='trough':
        
        # Find the -dimmest- magnitude in the specified phase range
        mag_feat = max(mag_search_np)
        
        index_feat_int = mag_search_np.argmax()
        
    #---------------------------------------------------
    #  Determine the index, day and e_mag of the feature.
        
    index_feat = index_feat_int + mytoolsSNe.day2index(phase_range[0])

    day_feat = mytoolsSNe.index2day(index_feat)

    e_mag_feat = lcgp_np[index_feat][2]
    
    return np.array([index_feat, day_feat, mag_feat, e_mag_feat])


In [8]:
##############################################################################80

---

#### Loop to read and extract the values of peaks and the trough

In [9]:
# Features to extract from the GP LC files

if band == 'Y':
    feature_list = ['peak', 'trough', 'peak']
    feature_range = [[-7,0], [8, 20], [20,40]]
    
elif band == 'J':
    feature_list = ['peak', 'trough', 'peak']
    feature_range = [[-7,0], [8, 22], [20,40]]
    
elif band == 'H':
    feature_list = ['peak', 'trough', 'peak']
    feature_range = [[-7,0], [6, 18], [18,35]]
    
elif band == 'K':
    feature_list = ['peak', 'trough', 'peak']
    feature_range = [[-7,0], [7, 19], [18,36]]

In [10]:
# Loop to read, extract and write the values of peaks and the trough

#-----------------------------------------------------------------------------80
# Open the text file to write the peaks and trough

textfile_1 = open(dir_save_output+'1_lc_features_%s_band.dat'%band,'w')
textfile_1.write('#         LC peaks and trough (%s band)\n'%band)

textfile_1.write('#\n')
textfile_1.write('# Gaussian-Processes light-curve files located at:\n')
textfile_1.write('%s\n'%dirfile)
    
# Define the header text
now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y.%m.%d (yyyy.mm.dd); %H:%M hrs.")
text_Date   = '# On date: %s\n'%text_timenow
text_Author = '# Data table created by: %s\n'%code_created_by
text_script = '# Script used: %s (version %s | last update: %s)\n'%(
    code_name, code_version, code_last_update)
text_line = '#'+'-'*57 + '60\n'

# Write the header text
textfile_1.write(text_line);
textfile_1.write(text_Author); textfile_1.write(text_Date); textfile_1.write(text_script);
textfile_1.write(text_line);

textfile_1.write('#                        1st_peak                               \
trough                                 2nd_peak\n')

textfile_1.write('#sn_name,                index,  day,      mag,       \
e_mag,    index,  day,      mag,        e_mag,   index,  day,      mag,        e_mag,')

# textfile_1.close()

#-----------------------------------------------------------------------------80

count_sn_good = 0

for i1 in range(len(list_sne)):

    # Read the file name:
    sn_filename = str(list_sne['f0'][i1])[2:-1]
    print(sn_filename)
    
    # Upload the data:
    GP_data = np.genfromtxt(dirfile+sn_filename)

    # Write the supernova name:
    textfile_1.write('\n%s,   '%sn_filename[:22])

    # Loop over features to find ('peak', 'trough', 'peak')
    for i2 in range(len(feature_list)):

        feat_int1 = find_peak_trough(GP_data, feature=feature_list[i2], 
                     phase_range=feature_range[i2])

        index_out = feat_int1[0]
        day_out = feat_int1[1]
        mag_out = feat_int1[2]
        emag_out = feat_int1[3]

        textfile_1.write('%-3i,  %5.1f,  %9.5f,  %9.5f,   '%(
            index_out, day_out, mag_out, emag_out))
        
    count_sn_good += 1

#---------------------------------------------------------60
textfile_1.write('\n'+text_line);
textfile_1.write('# %s SNe in this list.'%count_sn_good)

textfile_1.close()
print('\n# ---- %s SNe written ----'%count_sn_good)

sn1999ee__U_45_B_1_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2001bt__B_22_V_2_Others_J_GP_mean_sigma_Filled_norma.dat
sn2001cz__B_14_V_1_Others_J_GP_mean_sigma_Filled_norma.dat
sn2001el__U_20_B_4_Others_J_GP_mean_sigma_Filled_norma.dat
sn2002dj__U_13_B_1_Others_J_GP_mean_sigma_Filled_norma.dat
sn2005el__U_55_B_3_CfA_J_GP_mean_sigma_Filled_norma.dat
sn2005el_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2005eq__U_22_B_2_CfA_J_GP_mean_sigma_Filled_norma.dat
sn2005eq_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2005eu__B_16_V_2_CfA_J_GP_mean_sigma_Filled_norma.dat
sn2005ki_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2006ax_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2006bh_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2006D__U_24_B_22_CfA_J_GP_mean_sigma_Filled_norma.dat
sn2006et_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2006kf_dummytext_CSP_J_GP_mean_sigma_Filled_norma.dat
sn2006le__U_36_B_3_CfA_J_GP_mean_sigma_Filled_norma.dat
sn2006lf__U_3_B_24_CfA_J_GP_mean_sig

In [11]:
textfile_1.close(); textfile_1.close(); textfile_1.close(); 